In [1]:
import requests
import time
import os, json
import csv
import concurrent.futures
# from functools import partial
import pandas as pd

In [2]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "http://vllm-serving-service:8888/v1/completions"
# lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
request_payload_file = '/data/Magicoder_payloads.txt'
report_path = '/data/vLLM_5Concurrency_1gpu.csv'
df = pd.DataFrame(columns=['Prompt', 'Response', 'Response_status', 'Time (seconds)'])

In [3]:
payload = {"prompt": "Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams.", 
           "max_tokens": 200,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [4]:
def read_payloads(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            prompt = json.loads(line.strip())
            yield prompt['payload']

In [5]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        global df
        start_time = time.time()
        result, _= func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = {'Prompt':_,'Response':result.text, 'Response_status':result.status_code, 'Time (seconds)':f'{execution_time:.6f}'}
        df = df.append(data, ignore_index=True)
        print(f"Execution time: {execution_time:.6f} seconds \n\n")
        return result
    return calculate_execution_time

In [6]:
def save_report(file_path, df):
    print(df)
    df.to_csv(file_path, index=False)

In [7]:
@my_decorator
def inference(payload_data=None):
    headers = {
        'Content-Type': 'application/json'
    }
    try:
        response = requests.request("POST", url, headers=headers, json=payload_data)
        print(f"Prompt==> {payload_data['prompt']} \n {response.text}")
        return response, payload_data['prompt']
    except Exception as e:
        print(e)
        

In [ ]:
#Model Inference(Request)
if __name__ == "__main__":
    # Using the generator function to read lines 
    prompts = list(read_payloads(request_payload_file))
    
    global df
    for i in range(5):
        print(f"loop {i}+++++++++++++++++++++++++++++")
        with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
            futures = [executor.submit(inference, {**payload, 'prompt': prompt}) for prompt in prompts]
            concurrent.futures.wait(futures)
    save_report(report_path, df)

loop 0+++++++++++++++++++++++++++++
Prompt==> Problem: Write a function in python to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-dca8b71f45e14bb9ac5c9a73681a1360","object":"text_completion","created":453568,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef reverse_string(s):\n    return s[::-1]\n\n# Test the function\nprint(reverse_string(\"Hello, World!\"))  # Output: !dlroW ,olleH","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":31,"total_tokens":82,"completion_tokens":51}}
Execution time: 2.851506 seconds 


Prompt==> Problem: Write a function in python to check if a given string is a palindrome.Input: String s.Output: Boolean, indicating whether s is a palindrome. 
 {"id":"cmpl-9aac14c94ce24761bce12a763c2f75a5","object":"text_completion","created":453568,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/sna

Prompt==> Problem: Write a function in java to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-996509ccf985413891ef570c395525c6","object":"text_completion","created":453580,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\npublic class Main {\n    public static void main(String[] args) {\n        String s = \"Hello World\";\n        System.out.println(reverseString(s));\n    }\n\n    public static String reverseString(String s) {\n        String reversed = \"\";\n        for (int i = s.length() - 1; i >= 0; i--) {\n            reversed += s.charAt(i);\n        }\n        return reversed;\n    }\n}","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":31,"total_tokens":146,"completion_tokens":115}}
Execution time: 5.953223 seconds 


Prompt==> Problem: Write a function in java to calculate the factorial of a given non-negative integ

Prompt==> Problem: Write a function in java to perform binary search on a sorted list.Input: Sorted list arr and target value target.Output: Integer, the index of target in arr (or -1 if not found). 
 {"id":"cmpl-f9a0ff0df053494fbaa4491d9de6f8f1","object":"text_completion","created":453593,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\n```java\npublic class BinarySearch {\n    public static int binarySearch(int[] arr, int target) {\n        int left = 0;\n        int right = arr.length - 1;\n\n        while (left <= right) {\n            int mid = left + (right - left) / 2;\n\n            if (arr[mid] == target) {\n                return mid;\n            } else if (arr[mid] < target) {\n                left = mid + 1;\n            } else {\n                right = mid - 1;\n            }\n        }\n\n        return -1;\n    }\n\n    public static void main(String[] args) {\n        int[

Prompt==> Problem: Write a function in javascript to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-a6a9fdcb8ef14049b2a2f60bb396c806","object":"text_completion","created":453617,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nfunction reverseString(s) {\n    return s.split(\"\").reverse().join(\"\");\n}\n\nconsole.log(reverseString(\"Hello World\")); // Output: \"dlroW olleH\"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":31,"total_tokens":82,"completion_tokens":51}}
Execution time: 2.671413 seconds 


Prompt==> Problem: Write a function in javascript to calculate the factorial of a given non-negative integer.Input: Non-negative integer n. Output: Integer, the factorial of n. 
 {"id":"cmpl-51b4bbc592b54316b1ef6ef42b17efb0","object":"text_completion","created":453617,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.

Prompt==> Problem: Write a function in Ruby to calculate the factorial of a given non-negative integer.Input: Non-negative integer n. Output: Integer, the factorial of n. 
 {"id":"cmpl-c52aa069130d4b0593818740cbeff3f3","object":"text_completion","created":453632,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef factorial(n)\n  if n == 0\n    1\n  else\n    n * factorial(n-1)\n  end\nend\n\nputs factorial(5)","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":41,"total_tokens":88,"completion_tokens":47}}
Execution time: 2.418298 seconds 


Prompt==> Problem: Write a function in javascript to generate the nth Fibonacci number.Input: Non-negative integer n.Output: Integer, the nth Fibonacci number. 
 {"id":"cmpl-f91cb74e443d411eb178efcf77ef3717","object":"text_completion","created":453624,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c

...

Prompt==> Problem: Write a function in Golang to calculate the sum of the digits of a given positive integer.Input: Positive integer num.Output: Integer, the sum of the digits of num. 
 {"id":"cmpl-94f986cf3c8d43249e9adf389adfd215","object":"text_completion","created":453839,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple function in Golang to calculate the sum of the digits of a given positive integer:\n\n```go\npackage main\n\nimport (\n\t\"fmt\"\n\t\"strconv\"\n)\n\nfunc sumOfDigits(num int) int {\n\tsum := 0\n\tstrNum := strconv.Itoa(num)\n\tfor _, digit := range strNum {\n\t\tnum, _ := strconv.Atoi(string(digit))\n\t\tsum += num\n\t}\n\treturn sum\n}\n\nfunc main() {\n\tfmt.Println(sumOfDigits(1234)) // Output: 10\n}\n```\n\nThis function works by converting the integer to a string, then iterating over each character in the string. For each character, it converts it b

Prompt==> Problem: Build a function in CSS to toggle between light and dark modes for a website.Input: None.Output: None (update the website theme). 
 {"id":"cmpl-13985002c09743ca81653a51bb167924","object":"text_completion","created":453855,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple CSS function to toggle between light and dark modes:\n\n```css\n@media (prefers-color-scheme: dark) {\n  body {\n    background-color: #333;\n    color: #fff;\n  }\n}\n\n@media (prefers-color-scheme: light) {\n  body {\n    background-color: #fff;\n    color: #333;\n  }\n}\n```\n\nThis CSS function uses the `prefers-color-scheme` media feature to detect the user's preferred color scheme. If the user's preferred color scheme is dark, the body background color will be set to a dark grey (#333) and the text color will be set to white (#fff). If the user's preferred color scheme is light, the 

Prompt==> Problem: Create a function in CSS to toggle a specific class on an HTML element when a button is clicked.Input: HTML element selector, class name.Output: None (update the class of the HTML element). 
 {"id":"cmpl-2682d3e0c20c4d999e0b9832dd24afef","object":"text_completion","created":453871,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple CSS function that toggles a class on an HTML element when a button is clicked:\n\n```css\n@keyframes toggleClass {\n  0% {\n    opacity: 0;\n  }\n  100% {\n    opacity: 1;\n  }\n}\n\nbutton {\n  animation: toggleClass 2s;\n}\n\nbutton:hover {\n  background-color: #f0f0f0;\n}\n\n.myClass {\n  color: red;\n}\n\nbutton:active + .myClass {\n  color: blue;\n}\n```\n\nIn this example, when the button is clicked, the class \"myClass\" is toggled on the HTML element. The animation makes the button fade in over 2 seconds. The hover effect 

Prompt==> Problem: Build a function in react to populate an HTML table with data from an array of objects.Input: Array of objects, each representing a row in the table.Output: None (update the HTML table). 
 {"id":"cmpl-ac374419512e403b9891accbd72a595a","object":"text_completion","created":453886,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nHere is a simple example of how you can do this in React:\n\n```jsx\nimport React from 'react';\n\nclass Table extends React.Component {\n  constructor(props) {\n    super(props);\n    this.state = {\n      data: [\n        {id: 1, name: 'John', age: 25},\n        {id: 2, name: 'Jane', age: 30},\n        {id: 3, name: 'Bob', age: 20}\n      ]\n    };\n  }\n\n  render() {\n    return (\n      <table>\n        <thead>\n          <tr>\n            <th>ID</th>\n            <th>Name</th>\n            <th>Age</th>\n          </tr>\n        </thead>\n      

Prompt==> Problem: Create an animated dropdown menu in angular that appears when a button is clicked.Input: Button element selector, dropdown menu element selector.Output: None (animate the dropdown menu). 
 {"id":"cmpl-8ed75db12b5648688c09eeec740d12b6","object":"text_completion","created":453902,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nSolution:\n\nTo create an animated dropdown menu in Angular, you can use Angular's built-in animations feature. Here's a simple example:\n\nFirst, you need to import the BrowserAnimationsModule in your app.module.ts file:\n\n```typescript\nimport { BrowserAnimationsModule } from '@angular/platform-browser/animations';\n\n@NgModule({\n  imports: [\n    BrowserAnimationsModule\n  ],\n  // ...\n})\nexport class AppModule { }\n```\n\nThen, you can define the animation in your component's animations array:\n\n```typescript\nimport { trigger, state, style,

In [ ]:
# import aiohttp
# import asyncio
# import time

# start_time = time.time()
# headers={"Content-type":"application/json"} 
# async def main():

#     async with aiohttp.ClientSession(trust_env = True, headers=headers) as session:

# #         for payload in payloads:
# #             url='https://refract.fosfor.com/openchatfinetunedimqlgenerator/d0721c53-c9f5-4d79-a988-2d21df46198f/score' 
#         async with session.post(url, json=payload) as resp:
#                 mql = await resp.json(content_type=None)
#                 print(mql)

# await main()
# print("--- %s seconds ---" % (time.time() - start_time))